In [2]:
import duckdb

### Crear base de datos persistente


In [3]:

db = duckdb.connect("csar_data.db")

#inmem = duckdb.connect()
#conexion a memoria

## Carga de datos


In [ ]:
# create a new table from the contents of a DataFrame
db.execute("CREATE TABLE CATEGORY AS SELECT * FROM '../../files/CATEGORY.csv'")
db.execute("CREATE TABLE PRODUCT_CATEGORY AS SELECT * FROM '../../files/PRODUCT_CATEGORY.csv'")
db.execute("CREATE TABLE PRODUCT AS SELECT * FROM '../../files/PRODUCT.csv'")

## Pruebas de consulta

In [ ]:
# select * from CATEGORY
db.sql("select * from CATEGORY where category_name LIKE 'Pa%' ")

┌─────────────┬───────────────┐
│ category_id │ category_name │
│    int64    │    varchar    │
├─────────────┼───────────────┤
│           2 │ Panadería     │
└─────────────┴───────────────┘

## Common Table Expression  (CTE)

In [29]:
db.sql(
  '''
    WITH PRODUCT_CATEGORY_NAMES AS
    ( SELECT *
    FROM PRODUCT_CATEGORY PC
    LEFT JOIN CATEGORY C
    ON PC.category_id == C.category_id
    )
    SELECT * FROM PRODUCT_CATEGORY_NAMES

  '''
)

┌────────────┬─────────────┬───────────────┬───────────────┐
│ product_id │ category_id │ category_id_1 │ category_name │
│   int64    │    int64    │     int64     │    varchar    │
├────────────┼─────────────┼───────────────┼───────────────┤
│          1 │           6 │             6 │ Verduras      │
│          2 │           6 │             6 │ Verduras      │
│          3 │           5 │             5 │ Frutas        │
│          4 │           7 │             7 │ Cereales      │
│          5 │           8 │             8 │ Enlatados     │
│          6 │           6 │             6 │ Verduras      │
│          7 │           8 │             8 │ Enlatados     │
│          8 │           4 │             4 │ Carnes        │
│          9 │           9 │             9 │ Limpieza      │
│         10 │           5 │             5 │ Frutas        │
│          · │           · │             · │   ·           │
│          · │           · │             · │   ·           │
│          · │          

## JOINS

In [35]:
db.sql(
  '''
    SELECT PC.*,C.category_name
    FROM PRODUCT_CATEGORY PC
    JOIN CATEGORY C
    ON PC.category_id == C.category_id

  '''
)

┌────────────┬─────────────┬───────────────┐
│ product_id │ category_id │ category_name │
│   int64    │    int64    │    varchar    │
├────────────┼─────────────┼───────────────┤
│          1 │           6 │ Verduras      │
│          2 │           6 │ Verduras      │
│          3 │           5 │ Frutas        │
│          4 │           7 │ Cereales      │
│          5 │           8 │ Enlatados     │
│          6 │           6 │ Verduras      │
│          7 │           8 │ Enlatados     │
│          8 │           4 │ Carnes        │
│          9 │           9 │ Limpieza      │
│         10 │           5 │ Frutas        │
│          · │           · │   ·           │
│          · │           · │   ·           │
│          · │           · │   ·           │
│         91 │           9 │ Limpieza      │
│         92 │           8 │ Enlatados     │
│         93 │           4 │ Carnes        │
│         94 │          10 │ Dulces        │
│         95 │           5 │ Frutas        │
│         

In [ ]:
#data.avg('category_id_1')

┌────────────────────┐
│ avg(category_id_1) │
│       double       │
├────────────────────┤
│               6.34 │
└────────────────────┘

## Funciones Ventana (windows functions)

In [38]:
db.sql(
  '''
    SELECT PC.*,C.category_name
    FROM PRODUCT_CATEGORY PC
    JOIN CATEGORY C
    ON PC.category_id == C.category_id

  '''
)

┌────────────┬─────────────┬───────────────┐
│ product_id │ category_id │ category_name │
│   int64    │    int64    │    varchar    │
├────────────┼─────────────┼───────────────┤
│          1 │           6 │ Verduras      │
│          2 │           6 │ Verduras      │
│          3 │           5 │ Frutas        │
│          4 │           7 │ Cereales      │
│          5 │           8 │ Enlatados     │
│          6 │           6 │ Verduras      │
│          7 │           8 │ Enlatados     │
│          8 │           4 │ Carnes        │
│          9 │           9 │ Limpieza      │
│         10 │           5 │ Frutas        │
│          · │           · │   ·           │
│          · │           · │   ·           │
│          · │           · │   ·           │
│         91 │           9 │ Limpieza      │
│         92 │           8 │ Enlatados     │
│         93 │           4 │ Carnes        │
│         94 │          10 │ Dulces        │
│         95 │           5 │ Frutas        │
│         

## Descripcion de la consulta

Calcula cual es el segundo producto mas caro por categoria

In [45]:
db.sql(
  '''

    WITH PRODUCT_CATEGORY_NAMES AS
    ( SELECT *
    FROM PRODUCT_CATEGORY PC
    LEFT JOIN CATEGORY C
    ON PC.category_id == C.category_id
    )

    Select p.*,PCN.category_id, PCN.category_name
    from PRODUCT p
    JOIN PRODUCT_CATEGORY_NAMES PCN ON p.product_id == PCN.product_id

  '''
)

┌────────────┬──────────────┬─────────┬────────┬─────────┬─────────────┬───────────────┐
│ product_id │ product_name │  brand  │ price  │  unit   │ category_id │ category_name │
│   int64    │   varchar    │ varchar │ double │ varchar │    int64    │    varchar    │
├────────────┼──────────────┼─────────┼────────┼─────────┼─────────────┼───────────────┤
│          1 │ Arroz        │ MarcaA  │  54.62 │ paquete │           6 │ Verduras      │
│          2 │ Huevos       │ MarcaD  │  36.87 │ ml      │           6 │ Verduras      │
│          3 │ Manzana      │ MarcaC  │  86.13 │ ml      │           5 │ Frutas        │
│          4 │ Ajo          │ MarcaD  │   96.4 │ ml      │           7 │ Cereales      │
│          5 │ Pescado      │ MarcaC  │  76.84 │ ml      │           8 │ Enlatados     │
│          6 │ Refresco     │ MarcaB  │  93.83 │ paquete │           6 │ Verduras      │
│          7 │ Harina       │ MarcaB  │  43.21 │ l       │           8 │ Enlatados     │
│          8 │ Uva   

In [82]:
db.sql(
  '''
  select p.product_id,p.product_name,p.price,
  RANK() OVER(
    ORDER BY p.price ASC
  ) AS price_rank
  from PRODUCT p

  '''
)

┌────────────┬──────────────┬────────┬────────────┐
│ product_id │ product_name │ price  │ price_rank │
│   int64    │   varchar    │ double │   int64    │
├────────────┼──────────────┼────────┼────────────┤
│         24 │ Yogurt       │   3.39 │          1 │
│         52 │ Azúcar       │   3.41 │          2 │
│         67 │ Leche        │   5.45 │          3 │
│         29 │ Aceite       │   9.11 │          4 │
│         19 │ Arroz        │  10.27 │          5 │
│         95 │ Cereal       │  10.68 │          6 │
│         47 │ Tomate       │  12.75 │          7 │
│         49 │ Pan          │  15.38 │          8 │
│         10 │ Harina       │  16.12 │          9 │
│         14 │ Banana       │  17.31 │         10 │
│          · │   ·          │    ·   │          · │
│          · │   ·          │    ·   │          · │
│          · │   ·          │    ·   │          · │
│          6 │ Refresco     │  93.83 │         91 │
│         53 │ Manzana      │  94.66 │         92 │
│         21

In [88]:
db.sql(
  '''
    WITH PRODUCT_CATEGORY_NAMES AS
    ( SELECT *
    FROM PRODUCT_CATEGORY PC
    LEFT JOIN CATEGORY C
    ON PC.category_id == C.category_id
    )

    Select p.*,PCN.category_id, PCN.category_name,
    RANK() OVER(
    PARTITION BY PCN.category_id
    ORDER BY p.price ASC
    ) AS price_rank
    from PRODUCT p
    JOIN PRODUCT_CATEGORY_NAMES PCN ON p.product_id == PCN.product_id



  '''
)

┌────────────┬──────────────┬─────────┬────────┬─────────┬─────────────┬───────────────┬────────────┐
│ product_id │ product_name │  brand  │ price  │  unit   │ category_id │ category_name │ price_rank │
│   int64    │   varchar    │ varchar │ double │ varchar │    int64    │    varchar    │   int64    │
├────────────┼──────────────┼─────────┼────────┼─────────┼─────────────┼───────────────┼────────────┤
│         14 │ Banana       │ MarcaB  │  17.31 │ ml      │           7 │ Cereales      │          1 │
│         31 │ Uva          │ MarcaC  │  22.56 │ unidad  │           7 │ Cereales      │          2 │
│         83 │ Uva          │ MarcaA  │  40.41 │ unidad  │           7 │ Cereales      │          3 │
│         17 │ Frijoles     │ MarcaD  │  60.03 │ unidad  │           7 │ Cereales      │          4 │
│         50 │ Jugo         │ MarcaB  │  68.91 │ unidad  │           7 │ Cereales      │          5 │
│         54 │ Mantequilla  │ MarcaA  │  69.38 │ ml      │           7 │ Cereales 

In [92]:
db.sql(
  '''
    WITH PRODUCT_CATEGORY_NAMES AS
    ( SELECT *
    FROM PRODUCT_CATEGORY PC
    LEFT JOIN CATEGORY C
    ON PC.category_id == C.category_id
    )

    SELECT *
    from (
      Select p.*,PCN.category_id, PCN.category_name,
      RANK() OVER(
      PARTITION BY PCN.category_id
      ORDER BY p.price ASC
      ) AS price_rank
      from PRODUCT p
      JOIN PRODUCT_CATEGORY_NAMES PCN ON p.product_id == PCN.product_id

    ) as category_ranking
    where category_ranking.price_rank =2





  '''
)

┌────────────┬──────────────┬─────────┬────────┬─────────┬─────────────┬───────────────┬────────────┐
│ product_id │ product_name │  brand  │ price  │  unit   │ category_id │ category_name │ price_rank │
│   int64    │   varchar    │ varchar │ double │ varchar │    int64    │    varchar    │   int64    │
├────────────┼──────────────┼─────────┼────────┼─────────┼─────────────┼───────────────┼────────────┤
│         61 │ Tomate       │ MarcaE  │  80.46 │ l       │           1 │ Lácteos       │          2 │
│         47 │ Tomate       │ MarcaC  │  12.75 │ unidad  │           2 │ Panadería     │          2 │
│         68 │ Pescado      │ MarcaE  │  25.48 │ g       │           3 │ Bebidas       │          2 │
│         55 │ Naranja      │ MarcaC  │  23.64 │ g       │           6 │ Verduras      │          2 │
│         45 │ Manzana      │ MarcaD  │  40.09 │ unidad  │          10 │ Dulces        │          2 │
│         95 │ Cereal       │ MarcaE  │  10.68 │ g       │           5 │ Frutas   